In [1]:
from nlptoolkits.FirmLevelRiskKits import DictionaryT, PreprocessT, ScorerT
from nlptoolkits.SmallKits import IOHandlerT
import nlptoolkits
import pandas as pd
import tempfile

We make the basic classes for further use

In [2]:
internation_tax_dictionary = IOHandlerT.file_to_list(
    './input_data/annotatedsent_Principles_of_International_Taxation.txt',
    charset_error_encoding='utf-8'
)[:1000]

tweets_texture = IOHandlerT.file_to_list(
    './input_data/tweets_origin.txt',
    charset_error_encoding='utf-8'
)[:1000]

# make a class to clean the data(dict)
corenlp_preprocess_cls = PreprocessT.NgramDataPreprocessor(
    # remove_stopwords_set=nlptoolkits.resources.SET_STOPWORDS
)

# make a class to build the dictionary
corenlp_ngram_dict_cls = DictionaryT.NgramDictionaryBuilder(
    # remove_stopwords_set=nlptoolkits.resources.SET_STOPWORDS
)

In [3]:
internation_tax_dictionary[:3]

In [4]:
tweets_texture[:3]

Part to make Ngram dictionary

In [5]:
international_tax_tfidf = corenlp_ngram_dict_cls.n_gramizer_dictionary_builder(
    internation_tax_dictionary, scorer = 'tfidf', 
    final_remove_token_lessequal_then_length=1
    )
international_tax_tfidf

In [6]:
international_tax_tfidf

You can remove words by remove phrases which are in another dictionary, but we do not do here now.

We now clean the tweets data quickly by using the nltk quick cleaner

In [7]:
PreprocessT.naive_nltk_annotator(
    tweets_texture,
    processes=1
)[0]

In [8]:
tweets_texture_cleaned = corenlp_preprocess_cls.clean_annotated_texture_list(PreprocessT.naive_nltk_annotator(
    tweets_texture,
    processes=5
), processes=5, final_remove_token_lessequal_then_length=1)
tweets_texture_cleaned[0]

Actually you could also use the StanfordNLP cleaner to do so:

In [9]:
corenlp_preprocess_cls.alias_auto_clean_annotated_txt(
    processes=5,
    path_in_parsed_txt='./input_data/annotatedsent_Principles_of_International_Taxation.txt',
    path_out_cleaned_txt='./input_data/annotatedsent_Principles_of_International_Taxation_cleaned.txt',
    final_remove_token_lessequal_then_length=1
)

Then we can try to give a score. we have some already known subject set like "Risk"

In [10]:
nlptoolkits.resources.SET_OXFORD_SYNONYMS_RISK_WORDS_LOWER

In [11]:
scorer_cls = ScorerT.NgramScorer()

In [12]:
scorer_cls.scorer(
    tweets_texture_cleaned[2],
    topic_ngram_weighted_map_dict={'globtax':international_tax_tfidf},
    subject_word_set_dict={'risk':nlptoolkits.resources.SET_OXFORD_SYNONYMS_RISK_WORDS_LOWER,
                           'sentpos':nlptoolkits.resources.SET_LOUGHRAN_MCDONALD_POSITIVE_WORDS_LOWER,
                           'sentneg':nlptoolkits.resources.SET_LOUGHRAN_MCDONALD_NEGATIVE_WORDS_LOWER,
                           },
    is_scale_by_totalwords=True,
    binary_transformation_subjects=['sentpos'],
    scale_multiplier=100000
)

Batch run ->

In [14]:
rsts_tp = scorer_cls.list_scorer(
    tweets_texture_cleaned,
    topic_ngram_weighted_map_dict={'globtax':international_tax_tfidf},
    subject_word_set_dict={'risk':nlptoolkits.resources.SET_OXFORD_SYNONYMS_RISK_WORDS_LOWER,
                           'sentpos':nlptoolkits.resources.SET_LOUGHRAN_MCDONALD_POSITIVE_WORDS_LOWER,
                           'sentneg':nlptoolkits.resources.SET_LOUGHRAN_MCDONALD_NEGATIVE_WORDS_LOWER,
                           },
    is_scale_by_totalwords=True,
    binary_transformation_subjects=['risk'],
    scale_multiplier=100000
)

In [15]:
pd.DataFrame(rsts_tp)